In [16]:
%pip install pymysql

Note: you may need to restart the kernel to use updated packages.


In [26]:
import pandas as pd
from difflib import SequenceMatcher
import itertools
import pyarrow.parquet as pq

# Load Master Data
df= pd.read_csv('/Users/priyarajendran/Desktop/git/rajendran-priya/oubt-training/yellow-taxi-data-set /taxi_zone_lookup.csv')


# 1. Simple Matching: Identify rows with identical Zone and Borough names
# subset=['Zone', 'Borough'] tells pandas to only look at those two columns
# keep=False ensures we see ALL copies of the duplicate
exact_duplicates = df[df.duplicated(subset=['Zone', 'Borough'], keep=False)]

print("--- DUPLICATE RECORDS DETECTED ---")
print(exact_duplicates.sort_values(by='Zone'))

# 3. Cleansing: Keep only the first occurrence (The "Golden Record")
df_master_clean = df.drop_duplicates(subset=['Zone', 'Borough'], keep='first')

# 4. Save the results
df_master_clean.to_csv('cleaned_taxi_zones.csv', index=False)
print(f"\nOriginal count: {len(df)}")

# 5. Fuzzy Match: Compare all unique zone names
unique_zones = df_master_clean['Zone'].dropna().unique().tolist()
fuzzy_matches = []

for z1, z2 in itertools.combinations(unique_zones, 2):
    score = SequenceMatcher(None, z1, z2).ratio()
    # 0.85 is a high similarity threshold
    if 0.85 <= score < 1.0:
        fuzzy_matches.append((z1, z2, round(score, 3)))

# Output results
print("Exact Duplicates Found:", len(exact))
print("Top Fuzzy Matches:", fuzzy_matches[:5])

--- DUPLICATE RECORDS DETECTED ---
     LocationID    Borough                                           Zone  \
55           56     Queens                                         Corona   
56           57     Queens                                         Corona   
102         103  Manhattan  Governor's Island/Ellis Island/Liberty Island   
103         104  Manhattan  Governor's Island/Ellis Island/Liberty Island   
104         105  Manhattan  Governor's Island/Ellis Island/Liberty Island   

    service_zone  
55     Boro Zone  
56     Boro Zone  
102  Yellow Zone  
103  Yellow Zone  
104  Yellow Zone  

Original count: 265
Exact Duplicates Found: 5
Top Fuzzy Matches: [('Bensonhurst East', 'Bensonhurst West', 0.875), ('Bushwick North', 'Bushwick South', 0.857), ('Crown Heights North', 'Crown Heights South', 0.895), ('East Harlem North', 'East Harlem South', 0.882), ('Financial District North', 'Financial District South', 0.917)]


In [27]:


# 1. Load the "Golden" Master Data
master_df = df_master_clean
master_ids = set(master_df['LocationID'].unique())
trips = '/Users/priyarajendran/Desktop/git/rajendran-priya/oubt-training/yellow-taxi-data-set /yellow_tripdata_2025-08.parquet'

# 2. Load the Transaction Data (Parquet)
# We only need the Location columns to save memory
transactions = pd.read_parquet(
    trips, 
    columns=['PULocationID', 'DOLocationID']
)

# 3. Extract unique IDs from Transactions
pickup_ids = set(transactions['PULocationID'].unique())
dropoff_ids = set(transactions['DOLocationID'].unique())
all_transaction_ids = pickup_ids.union(dropoff_ids)

# 4. FIND THE ORPHANS (In Transactions but NOT in Master)
orphans = all_transaction_ids - master_ids

# 5. FIND UNUSED MASTER DATA (In Master but NOT in Transactions)
unused = master_ids - all_transaction_ids

print("--- REFERENTIAL INTEGRITY REPORT ---")
if not orphans:
    print("✅ SUCCESS: No orphaned records found. All trips map to valid zones.")
else:
    print(f"❌ ALERT: Found {len(orphans)} orphaned LocationIDs!")
    print(f"Orphaned IDs: {orphans}")
    
print(f"\nTotal Unique Zones in Transactions: {len(all_transaction_ids)}")
print(f"Total Golden Records in Master: {len(master_ids)}")
print(f"Unused Master Zones (No trips recorded): {len(unused)}")

--- REFERENTIAL INTEGRITY REPORT ---
❌ ALERT: Found 1 orphaned LocationIDs!
Orphaned IDs: {57}

Total Unique Zones in Transactions: 261
Total Golden Records in Master: 262
Unused Master Zones (No trips recorded): 2


In [28]:
import pandas as pd

# 1. Load your Clean Master Data
master = master_df

# 2. Load your Transaction Data
trips_df = pd.read_parquet(trips)

# 3. The Remapping Fix:
# In our check, we saw ID 57 was an orphan because we kept 56.
# We replace the 'Retired' ID with the 'Golden' ID.
mapping = {57: 56, 104: 103, 105: 103}

trips_df['PULocationID'] = trips_df['PULocationID'].replace(mapping)
trips_df['DOLocationID'] = trips_df['DOLocationID'].replace(mapping)

# 4. Save the "Golden" Transactions
trips_df.to_parquet('Final_Golden_Transactions.parquet')

print("Success: All transactions now point to Golden Master IDs.")

Success: All transactions now point to Golden Master IDs.


In [ ]:
yellotaxi-master-database.cgvogeskmafq.us-east-1.rds.amazonaws.com

In [ ]:

from sqlalchemy import create_engine, text

# 1. Connection setup
engine = create_engine("mysql+pymysql://raajendran.priya@gmail.com:Check123*@yellotaxi-master-database.cgvogeskmafq.us-east-1.rds.amazonaws.com")



# Extract unique Boroughs and Service Zones for lookup tables
boroughs = pd.DataFrame(master_df['Borough'].unique(), columns=['BoroughName']).reset_index(names='BoroughID')
service_zones = pd.DataFrame(master_df['service_zone'].unique(), columns=['ServiceName']).reset_index(names='ServiceID')

# Map the IDs back to create the Master table
df_master = master_df.merge(boroughs, left_on='Borough', right_on='BoroughName')
df_master = df_master.merge(service_zones, left_on='service_zone', right_on='ServiceName')
print(df_master.head())
df_master = df_master[['LocationID', 'Zone', 'BoroughID', 'ServiceID']]

# 3. Load to RDS
with engine.begin() as conn:
    # Create tables
    boroughs.to_sql('dim_borough', conn, if_exists='replace', index=False)
    service_zones.to_sql('dim_service_zone', conn, if_exists='replace', index=False)
    df_master.to_sql('dim_taxi_zones', conn, if_exists='replace', index=False)

    # 4. ENFORCE GOVERNANCE (Integrity Constraints)
    conn.execute(text("ALTER TABLE dim_taxi_zones ADD PRIMARY KEY (LocationID);"))
    conn.execute(text("ALTER TABLE dim_borough ADD PRIMARY KEY (BoroughID);"))
    conn.execute(text("ALTER TABLE dim_service_zone ADD PRIMARY KEY (ServiceID);"))
    
    # Add Foreign Keys to ensure no 'orphaned' metadata
    conn.execute(text("""
        ALTER TABLE dim_taxi_zones 
        ADD CONSTRAINT fk_borough FOREIGN KEY (BoroughID) REFERENCES dim_borough(BoroughID),
        ADD CONSTRAINT fk_service FOREIGN KEY (ServiceID) REFERENCES dim_service_zone(ServiceID);
    """))

print("✅ Master Data Tables built and governed in RDS.")

   LocationID        Borough                     Zone service_zone  BoroughID  \
0           1            EWR           Newark Airport          EWR          0   
1           2         Queens              Jamaica Bay    Boro Zone          1   
2           3          Bronx  Allerton/Pelham Gardens    Boro Zone          2   
3           4      Manhattan            Alphabet City  Yellow Zone          3   
4           5  Staten Island            Arden Heights    Boro Zone          4   

     BoroughName  ServiceID  ServiceName  
0            EWR          0          EWR  
1         Queens          1    Boro Zone  
2          Bronx          1    Boro Zone  
3      Manhattan          2  Yellow Zone  
4  Staten Island          1    Boro Zone  


OperationalError: (pymysql.err.OperationalError) (1045, "Access denied for user 'raajendran.priya@gmail.com'@'96.73.85.130' (using password: YES)")
(Background on this error at: https://sqlalche.me/e/20/e3q8)